In [ ]:

import numpy as np
import numba
from cil.framework import ImageGeometry, AcquisitionGeometry, DataOrder
from cil.utilities.display import show2D, show_geometry, show1D
from cil.framework import AcquisitionData
from cil.recon import FDK
from cil.optimisation.functions import TotalVariation, L2NormSquared, WeightedL2NormSquared, LeastSquares, MixedL21Norm, L1Norm
from cil.plugins.tigre import ProjectionOperator
from cil.optimisation.algorithms import PDHG, FISTA
from cil.optimisation.functions import BlockFunction, Function, IndicatorBox
from cil.optimisation.operators import BlockOperator, GradientOperator
from cil.utilities.quality_measures import mse
from cil.framework import ImageData
from cil.processors import  Padder, Slicer
import matplotlib.pyplot as plt
import json
import os
from cil.io import NEXUSDataWriter
import sys
import util
from CIL_Wavelets import WaveletOperator, WaveletNorm

# set up default colour map for visualisation
cmap = "gray"

# set the backend for FBP and the ProjectionOperator
device = 'gpu'

In [ ]:
n_subs = 36
gamma = .1
epochs = 20
alpha = 10
rho=.99

amin = 0
amax = 1.5

bbox = [150,230, 120,200]
vertical_slice = 170
horizontal_y = 180

dose = 'low'
image_number='0000'
filename = f'/opt/data/ICASSP24/train/train/'+image_number+f'_sino_{dose}_dose.npy'
clean_image_file=f'/opt/data/ICASSP24/train/train/'+image_number+'_clean_fdk_256.npy'

data=np.asarray(np.load(filename,allow_pickle=True), dtype=np.float32)
ground_truth=np.asarray(np.load(clean_image_file,allow_pickle=True), dtype=np.float32)


In [ ]:
# Set up the geometry needed for the data

In [ ]:
image_size = [300, 300, 300]
image_shape = [256, 256, 256]
voxel_size = [1.171875, 1.171875, 1.171875]

detector_shape = [256, 256]
detector_size = [600, 600]
pixel_size = [2.34375, 2.34375]

distance_source_origin = 575
distance_source_detector = 1050

#  S---------O------D
#

angles = np.linspace(0, 2*np.pi, 360, endpoint=False)